In [ ]:
'''
CODE BY NATE YEALANDS-FLINT
25051591@STUDENTS.LINCOLN.AC.UK
'''

In [ ]:
# get spark up and running (do this once when you restart a colab session or it times out)

# first, install the jdk
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# get spark itself
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz

# unzip spark
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
# install findspark so colab notebook sees it
!pip install -q findspark

# and setup env variable so pyspark works
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

# for using spark UI in colab... this allows tunnel to the UI
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

# Now spark is installed with correct env paths.. so use findspark to install it in colab
import findspark
findspark.init()

# start pyspark session
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType



from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()
# make the spark UI work in colab

get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels
spark

!pip install pandas

import pandas as pd
from functools import reduce
from multiprocessing import Pool, cpu_count
from collections import Counter
from timeit import default_timer as timer
import datetime

--2023-01-16 20:59:16--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 52.202.168.65, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  60.5MB/s    in 0.2s    

2023-01-16 20:59:17 (60.5 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
{"tunnels":[],"uri":"/api/tunnels"}
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#create columns for data files
AirportSchema = ["AirportName","IATA", "Latitude","Longitude"]
CSVSchema = ["PassengerID","FlightID","From","Dest", "DeptTime","FlightTime"]

#read files
PassengerData = pd.read_csv("AComp_Passenger_data_no_error.csv",encoding = 'latin1', names = CSVSchema)
Airports = pd.read_csv("Top30_airports_LatLong.csv",encoding = 'latin1', names = AirportSchema) 
Airports.dropna(inplace=True)

#create spark dataframe for easier use later
DT = spark.createDataFrame(PassengerData)

In [ ]:
def mapper(data): #create mapper
  splitdat = data.split()
  return Counter(splitdat)

def reducer(cnt1,cnt2): #create reducer
  cnt1.update(cnt2)
  return cnt1

##Count flights from airports, including ones not used

In [ ]:
#create lists to parallelise
Fromdata = PassengerData['From'].tolist()
IATAdata = Airports['IATA'].tolist()
AirNames = Airports['AirportName'].tolist()


Aerodat = {} #create dictionary with IATAs as the keys and airport names as the values
for i in range (0,len(IATAdata)):
  Aerodat[IATAdata[i]] = AirNames[i] 

In [ ]:
rddFrom = spark.sparkContext.parallelize(Fromdata,10)
Fromdata_mapped = rddFrom.map(mapper)
data_reduced = Fromdata_mapped.reduce(reducer) # get the count of airports that do get used
finalAir = dict(data_reduced)
AirList = [[],[],[]] #list so airport names can be included later

for i in IATAdata: #goes through IATA codes
  if i not in Fromdata: #if IATA code isn't in the list of start airports
    finalAir[i] = 0 #value in dictionary with the IATA as its key is 0
finalAir

AirList[0] = list(finalAir.keys()) # IATAs for first entry
AirList[2] = list(finalAir.values()) # Flight count for final entry
for i in range (0,len(AirList[0])): 
  AirList[1].append(Aerodat.get(AirList[0][i])) # get airport names

Air = pd.DataFrame(AirList) #creates dataframe to make the output nicer to read
Air #print airport names

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,DEN,JFK,ORD,KUL,MAD,LHR,CGK,MUC,AMS,DFW,...,PEK,HND,LAX,FRA,HKG,DXB,SIN,SFO,PHX,IST
1,DENVER,NEW YORK,CHICAGO,KUALA LUMPUR,MADRID,LONDON,JAKARTA,MUNICH,AMSTERDAM,DALLAS/FORT WORTH,...,BEIJING,TOKYO,LOS ANGELES,FRANKFURT,HONG KONG,DUBAI,SINGAPORE,SAN FRANCISCO,PHOENIX,ISTANBUL
2,46,25,33,33,13,25,27,14,15,11,...,13,13,0,0,0,0,0,0,0,0


##Calculate the Flight Passenger Counts, Date/Time etc

In [ ]:
Flightdata = PassengerData['FlightID'].tolist() # create list to parallelise
rddFlight =  spark.sparkContext.parallelize(Flightdata,10)
Flightdata_mapped = rddFlight.map(mapper)
data_reduced = Flightdata_mapped.reduce(reducer)
data_reduced

dictdata_reduced= dict(data_reduced)
flightInformation = []


for i in range(0, len(dictdata_reduced)):
  flightID = list(dictdata_reduced)[i] #add flight ID to list
  numPassengers = list(dictdata_reduced.values())[i] #Add passenger count to list
  for j in range (0, DT.count()):
    if (str(DT.collect()[j][1]) == str(list(dictdata_reduced)[i])):
      fromCode = DT.collect()[j][2] #get flight from
      toCode = DT.collect()[j][3] # get flight dest
      epochDepartureTime = DT.collect()[j][4] # get dept time
      totalFlightTime = DT.collect()[j][5] #get the arrival time
      #convert times
      departureTime = datetime.datetime.fromtimestamp(epochDepartureTime).strftime('%H:%M')
      arrivalTime = datetime.datetime.fromtimestamp(epochDepartureTime + (totalFlightTime*60)).strftime('%H:%M') 
      break
  flightInformation.append([flightID, numPassengers, fromCode, toCode, departureTime, arrivalTime])

FlightInfo = pd.DataFrame(flightInformation) #create dataframe for aesthetics of output
FlightInfo

,0,1,2,3,4,5
0,SQU6245R,21,DEN,FRA,17:14,10:43
1,XXQ4064B,25,JFK,FRA,17:05,06:27
2,SOH3431A,18,ORD,MIA,17:00,21:10
3,PME8178S,18,DEN,PEK,17:13,15:15
4,MBA8071P,16,KUL,PEK,17:04,02:36
5,MOO1786A,13,MAD,FRA,16:56,20:00
6,HUR0974O,7,DEN,PVG,17:15,16:33
7,GMO5938W,25,LHR,PEK,17:11,10:48
8,DAU2617A,12,CGK,SFO,17:23,23:34
9,RUM0422W,14,MUC,MAD,16:58,20:12


## Find the passenger with the most miles

In [ ]:
AirportsTask3 = Airports[['IATA','Latitude','Longitude']].copy() #copy columns into pandas
AirportsTask3 = spark.createDataFrame(AirportsTask3) # create dataframe from pandas 

PassData = PassengerData['PassengerID'].tolist() #data to parallelise

In [ ]:
from math import cos, sin, acos, pi


''' Nautical Miles = 3963 * acos( sin(ALat) * sin(BLat) + cos(ALat) * cos(BLat) * cos(BLong – ALong))

3963 is the radius of earth'''
def NautMileCalc(ALat,ALong,BLat,BLong):
  #convert to radians - calculation only works in radians
  ALat *= pi/180
  ALong *= pi/180 
  BLat *= pi/180
  BLong *= pi/180
  return 3963 * acos(sin(ALat) * sin(BLat) + cos(ALat) * cos(BLat) * cos(BLong - ALong)) #return calculation of nautical miles

def LongLatCalc(fromCode,toCode):
  for i in range(0,AirportsTask3.count()): # go through the dataframe with the longitudes and latitudea
    if AirportsTask3.collect()[i][0] == fromCode: #if the codes match, set the start longitude and latitudes 
      fromLat = AirportsTask3.collect()[i][1]
      fromLong = AirportsTask3.collect()[i][2]
    if AirportsTask3.collect()[i][0] == toCode: #if codes match, set end long/lat
      toLat = AirportsTask3.collect()[i][1]
      toLong = AirportsTask3.collect()[i][2]
  return NautMileCalc(fromLat,fromLong,toLat,toLong) #return the nautical mile calculation

In [ ]:
rddPass = spark.sparkContext.parallelize(PassData)
Passdata_mapped = rddPass.map(mapper)
Passdata_reduced = Passdata_mapped.reduce(reducer)

Passreduced = dict(Passdata_reduced)

PassengerInfo = [] # create array to store data in
for i in range(0, len(Passreduced)):
  PassengerID = list(Passreduced)[i] #set firsst value to passenger ID
  numFlights = list(Passreduced.values())[i] #set this to number of flights they've taken 
  currGroup = DT.filter(DT.PassengerID == str(list(Passreduced)[i])) #filter by passengerID
  earntMiles = 0
  for j in range (0, currGroup.count()): #while the number of flights taken hasn't been met
    earntMiles += LongLatCalc(currGroup.collect()[j][2],currGroup.collect()[j][3]) #calc nautical miles taken
    break
  PassengerInfo.append([PassengerID, numFlights,earntMiles]) #append this to list

#Compare sizes of passenger miles
Biggest = None
for Passenger in PassengerInfo:
  if Biggest == None:
    Biggest = Passenger #Set that passenger as the one with the most miles
  elif Biggest[2] < Passenger[2]:
    Biggest = Passenger
print(Biggest)

['KKP5277HZ7', 11, 9245.151195554365]
